In [127]:
import json, re
import numpy as np
from nltk.stem import WordNetLemmatizer

# User query on single recipe

In [128]:
wordnet_lemmatizer = WordNetLemmatizer()

In [160]:
with open('recipes/shepherds_pie.txt', 'r') as recipe_file: # INPUT: recipe file .txt
    recipe = recipe_file.read()
    print(recipe)

Peel the potatoes and cut into 1/2-inch dice. Place in a medium saucepan and cover with cold water. Set over high heat, cover and bring to a boil. Once boiling, uncover, decrease the heat to maintain a simmer and cook until tender and easily crushed with tongs, approximately 10 to 15 minutes. Place the half-and-half and butter into a microwave-safe container and heat in the microwave until warmed through, about 35 seconds. Drain the potatoes in a colander and then return to the saucepan. Mash the potatoes and then add the half and half, butter, salt and pepper and continue to mash until smooth. Stir in the yolk until well combined.

Preheat the oven to 400 degrees F.

While the potatoes are cooking, prepare the filling. Place the canola oil into a 12-inch saute pan and set over medium high heat. Once the oil shimmers, add the onion and carrots and saute just until they begin to take on color, approximately 3 to 4 minutes. Add the garlic and stir to combine. Add the lamb, salt and peppe

In [184]:
numberof_segments_after = 2 # INPUT: how many segments to read to user
question = "peel potatoes?" # INPUT: user question
question_lower_str = question.lower()
recipe_segments_lower_list = [segment.lower() for segment in re.split('; |\. |\n', recipe)]

max_similarity = 0
answer = ''
for idx, segment_lower_str in enumerate(recipe_segments_lower_list):
    similarity_score = 0
    segment_lower_nopunct_lemma_list = [wordnet_lemmatizer.lemmatize(re.sub(r'[^\w\s]','', seg_word), 'v') for seg_word in segment_lower_str.split()]
    for question_word_lower in question_lower_str.split():
        question_word_lower_nopunct = re.sub(r'[^\w\s]','', question_word_lower)
        question_word_lower_nopunct_lemma = wordnet_lemmatizer.lemmatize(question_word_lower_nopunct, 'v')
        if question_word_lower_nopunct_lemma in segment_lower_nopunct_lemma_list:
            similarity_score += 1
    if similarity_score > max_similarity:
        answer = segment_lower_str
        max_similarity = similarity_score
        for i in range(1, numberof_segments_after+1):
            if idx + i > len(recipe_segments_lower_list) - 1:
                print('breaking!', idx + i)
                break
            answer += '. {}'.format(recipe_segments_lower_list[idx+i])
print('MAX SIMILARITY:', max_similarity)
print(answer)

MAX SIMILARITY: 2
peel the potatoes and cut into 1/2-inch dice. place in a medium saucepan and cover with cold water. set over high heat, cover and bring to a boil


# BiDAF (ignore this for now)

In [22]:
query_to_json = {'passage': recipe,
                 'question': "What's after preheating the oven to 375?"}
with open('query.jsonl', 'w') as outfile:
    json.dump(query_to_json, outfile)
    outfile.write('\n')

a = time.time()
run = subprocess.run('python3 -m allennlp.run predict pretrained/bidaf-model-2017.09.15-charpad.tar.gz query.jsonl --output-file query_output.json'.split())
print('BiDAF finished after {} seconds'.format(time.time()-a))
# print(run)

with open('query_output.json', 'r') as f:
    for row in f:
        data = json.loads(row)
        print('ANSWER:', data['best_span_str'])
        break

BiDAF finished after 5.735774993896484 seconds
CompletedProcess(args=['python3', '-m', 'allennlp.run', 'predict', 'pretrained/bidaf-model-2017.09.15-charpad.tar.gz', 'query.jsonl', '--output-file', 'query_output.json'], returncode=0)
Preheat
